In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset = pd.read_csv(
                            'train-data.tsv', sep='\t',
                            names=['class', 'text']
                            )
dev_dataset = pd.read_csv(
                            'valid-data.tsv', sep='\t',
                             names=['class', 'text']
                             )

print(train_dataset.head())
print("-"*60)
print(dev_dataset.head())

In [ ]:
train_dataset['class'] = train_dataset['class'].map({'ham': 0, 'spam': 1})
dev_dataset['class'] = dev_dataset['class'].map({'ham': 0, 'spam': 1})

train_data, train_labels = train_dataset['text'], train_dataset['class']
dev_data, dev_labels = dev_dataset['text'], dev_dataset['class']

In [ ]:
train_word_counts = [len(message.split()) for message in train_data]
dev_word_counts = [len(message.split()) for message in dev_data]

max_words_train_dev = [max(train_word_counts), max(dev_word_counts)]
print(max_words_train_dev)

In [ ]:
train_message_len = [len(message) for message in train_data]
dev_message_len = [len(message) for message in dev_data]

max_seq_len_train_dev = [max(train_message_len), max(dev_message_len)]
print(max_seq_len_train_dev)

In [ ]:
max_words=max(max_words_train_dev)
max_seq_len=max(max_seq_len_train_dev)

# tokenize text
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data)

# convert text to int
train_seq = tokenizer.texts_to_sequences(train_data)
dev_seq = tokenizer.texts_to_sequences(dev_data)

# pad to have same lengths
train_seq_padded = keras.preprocessing.sequence.pad_sequences(train_seq, maxlen=max_seq_len)
dev_seq_padded = keras.preprocessing.sequence.pad_sequences(dev_seq, maxlen=max_seq_len)

In [ ]:
model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_words, output_dim=128),
    keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_seq_padded, train_labels, epochs=5, batch_size=32, validation_data=(dev_seq_padded, dev_labels))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    message_seq = tokenizer.texts_to_sequences([pred_text])
    message_padded = keras.preprocessing.sequence.pad_sequences(message_seq, maxlen=max_seq_len)

    proba = model.predict(message_padded)[0][0]

    predicted_label = 'ham' if proba < 0.5 else 'spam'

    return [proba, predicted_label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
